# 1. 웹 크롤링 로직

### 1.1 드라이버 기동 -> 홈페이지 이동 -> 빅뱅 검색 -> for문을 사용해서 12시즌 타이틀 넙버, 타이틀, 방영날짜, 평점 리스트에 넣기

In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service

season_list = []

n = 0

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service('../driver/chromedriver'), options=chrome_options)
driver.get('https://www.imdb.com/')

about = driver.find_element(By.CSS_SELECTOR, '#suggestion-search')

#css_selector = '''#docsearch-input'''
#search_docs = driver.find_element(By.CSS_SELECTOR, css_selector)
about.click()
about.send_keys('Big Bang Theory')

about.send_keys(Keys.ENTER)

name = driver.find_element(By.CSS_SELECTOR, '#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-78242121-0.iaCeoy > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.ffAEHI > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div.ipc-metadata-list-summary-item__tc > a')
name.click()
all_season = driver.find_element(By.CSS_SELECTOR, '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-9b716f3b-0.xBaTH > section > div:nth-child(4) > section > section > div.sc-11c0820-1.eDbIBR > div > div:nth-child(1) > a > span.sc-89e7233a-1.kYAXOp.episode-guide-text')
all_season.click()



for n in range(12):
    select_season = driver.find_element(By.ID, 'bySeason').find_elements(By.TAG_NAME, 'option')
    select_season[n].click()
    
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#episodes_content > div.clear > div.list.detail.eplist > div:nth-child(1) > div.info > strong > a')))
    print(n)
    
    detail = driver.find_element(By.CSS_SELECTOR, '#episodes_content > div.clear > div.list.detail.eplist')
    detail_boxes = detail.find_elements(By.CLASS_NAME, 'info')
    season_list.append([])
    for detail_box in detail_boxes:
        title = detail_box.find_element(By.TAG_NAME, 'strong').text
        airdate = detail_box.find_element(By.CLASS_NAME, 'airdate').text
        rate = detail_box.find_element(By.CLASS_NAME, 'ipl-rating-star__rating').text
        
        
        season_list[n].append([title,airdate,rate])


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-9b716f3b-0.xBaTH > section > div:nth-child(4) > section > section > div.sc-11c0820-1.eDbIBR > div > div:nth-child(1) > a > span.sc-89e7233a-1.kYAXOp.episode-guide-text"}
  (Session info: chrome=109.0.5414.119)
Stacktrace:
#0 0x5571cbc87303 <unknown>
#1 0x5571cba5bd37 <unknown>
#2 0x5571cba985b2 <unknown>
#3 0x5571cba986c1 <unknown>
#4 0x5571cbad2b34 <unknown>
#5 0x5571cbab89ad <unknown>
#6 0x5571cbad088c <unknown>
#7 0x5571cbab8753 <unknown>
#8 0x5571cba8ba14 <unknown>
#9 0x5571cba8cb7e <unknown>
#10 0x5571cbcd632e <unknown>
#11 0x5571cbcd9c0e <unknown>
#12 0x5571cbcbc610 <unknown>
#13 0x5571cbcdac23 <unknown>
#14 0x5571cbcae545 <unknown>
#15 0x5571cbcfb6a8 <unknown>
#16 0x5571cbcfb836 <unknown>
#17 0x5571cbd16d13 <unknown>
#18 0x7faa96e56609 start_thread


### 1.2 데이터 내부 확인

In [ ]:
season_list

In [ ]:
import pandas as pd

title_num = []
title_list =[]
airdate_list = []
rate_list = []


for i in range(len(season_list)):
    print(i)
    for j in range(len(season_list[i])):
        print(len(season_list[i]))
        title_num.append(f'{i+1}_{j+1}')
        title_list.append(season_list[i][j][0])
        airdate_list.append(season_list[i][j][1])
        rate_list.append(season_list[i][j][2])

In [ ]:
bigbang = pd.DataFrame({'title_num':title_num, 'title':title_list, 'airdate':airdate_list, 'rate':rate_list})
bigbang.set_index('title_num')
bigbang.head()

bigbang.sort_values(by='rate', ascending=False).head()

In [ ]:
num = []
title = []
airdate = []
rate = []
count = 1

for n in range(12):
    for each in season_list:
        num.append(count)
        count+=1
        title.append(each[n][0])
        airdate.append(each[n][1])
        rate.append(each[n][2])


In [ ]:
import numpy as np
samples = [1, 1, 2, 6, 7, 8, 9]
np.median(samples)

In [ ]:
a = [1,2,3,4,5,6,7,8,9,10]
a.pop()
print(a)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.pairplot(
    bigbang,
    x_vars=["airdate"],
    y_vars=["title_num"],
    hue="rate"
)

plt.show();

#plt.figure(figsize=(10,8))
#sns.heatmap(bigbang_pivot, annot=True, fmt="d")
#plt.show()

In [ ]:
#bigbang.sort_values(by='rate', ascending=False).head(20)

sns.jointplot(x=bigbang['rate'], y=bigbang['title_num'], kind = 'scatter')
